In [ ]:
def limpieza_chat(txt, nombre_archivo = None,excel = None, csv = None):
    if txt:
        import re
        import numpy as np
        import pandas as pd
        
        with open(txt, 'r', encoding='utf-8-sig', errors='ignore') as f:
            texto = f.read()
        
        # Reemplazar caracteres invisibles y normalizar espacios
        texto = texto.replace('\u200e', '').replace('\ufeff', '').replace('\xa0', ' ').replace('\u202f', ' ').replace('\u2009', ' ')
        lineas = texto.splitlines()
        
        # Patrón: [dd/mm/aa, hh:mm(:ss)? ?[ap]. m.]
        # Admite segundos opcionales y espacio no estándar entre la hora y “p. m.”
        patron_inicio = re.compile(r'^\[\d{1,2}/\d{2}/\d{2}, \d{1,2}:\d{2}(?::\d{2})?(?:\s?[ap]\.\sm\.)\]')
        
        mensajes_completos = []
        mensaje_actual = ""
        
        for linea in lineas:
            if patron_inicio.match(linea):
                if mensaje_actual:
                    mensajes_completos.append(mensaje_actual.strip())
                mensaje_actual = linea
            else:
                mensaje_actual += ' ' + linea.strip()
        
        if mensaje_actual:
            mensajes_completos.append(mensaje_actual.strip())

        fechas, horas, remitentes, mensajes = [], [], [], []
        
        for linea in mensajes_completos:
            fecha = re.search(r'\[(\d{1,2}/\d{2}/\d{2})', linea)
            hora = re.search(r', (.+?)\]', linea)
            remitente = re.search(r'\] ([^:]+):', linea)
            mensaje = re.search(r': (.+)', linea, re.DOTALL)
        
            fechas.append(fecha.group(1) if fecha else "")
            horas.append(hora.group(1).replace('\u202f', ' ') if hora else "")
            remitentes.append(remitente.group(1) if remitente else "")
            mensajes.append(mensaje.group(1).strip() if mensaje else np.nan)
        
        # Crear DataFrame
        df = pd.DataFrame({
            'Fecha': fechas,
            'Hora': horas,
            'Remitente': remitentes,
            'Mensaje': mensajes
        })

        df['Mensaje'] = df['Mensaje'].astype(str).str.lower()

        df['Hora'] = df['Hora'].str.replace('.', '', regex=False) \
                                    .str.replace(' ', '', regex=False) \
                                    .str.upper()


        df['Hora'] = pd.to_datetime(df['Hora'], format='%I:%M:%S%p') \
                 .dt.strftime('%H:%M:%S')

        df['Remitente'] = df['Remitente'].str.replace('Mi Niña❤️','Angela', regex = False) \
                                            .str.replace('Marcello Anchante','Marcello', regex = False)

        df['Fecha'] = pd.to_datetime(df['Fecha'], format="%d/%m/%y", errors='coerce')

        print(f'Datos correctamentes limpiados. \n El chat tiene {len(df)} mensajes')

        if nombre_archivo is not None:
            if excel is True:
                df.to_excel(f'data/{nombre_archivo}.xlsx', index = False)
            if csv is True:
                df.to_csv(f'data/{nombre_archivo}.csv', index = False, encoding = 'utf-8-sig')
        
        print(f'Datos correctamentes limpiados. \n El chat tiene {len(df)} mensajes')


            
        return df



In [2]:
row_data = limpieza_chat(txt = 'data/_chat.txt', nombre_archivo= 'chat_amor', excel = True, csv = True)

Datos correctamentes limpiados. 
 El chat tiene 48046 mensajes
Datos correctamentes limpiados. 
 El chat tiene 48046 mensajes


In [3]:
def skip_words_phase_one(df, columna, nombre_archivo = None,excel = None, csv = None):
    import numpy as np
    palabras_omitir = ['sticker omitido',
                   'llamada perdida  tocar para volver a llamar',
                   'videollamada  sin respuesta',
                   'imagen omitida',
                   'video omitido',
                   'sticker omitido',
                   'videollamada perdida  tocar para volver a llamar',
                   'los mensajes y las llamadas están cifrados de extremo a extremo. solo las personas en este chat pueden leerlos, escucharlos o compartirlos.',
                   'mi niña❤️ es un contacto.',
                   'audio omitido']

    df[columna] = df[columna].apply(
        lambda x: np.nan if any(palabra in x for palabra in palabras_omitir) else x
    )

    if nombre_archivo is not None:
        if excel is True:
            df.to_excel(f'data/{nombre_archivo}.xlsx', index = False)
        if csv is True:
            df.to_csv(f'data/{nombre_archivo}.csv', index = False, encoding = 'utf-8-sig')

    return df


In [4]:
cleaned_data = skip_words_phase_one(df = row_data, columna = 'Mensaje', nombre_archivo = 'chat_limpio', excel = True, csv = True)

In [5]:
cleaned_data

,Fecha,Hora,Remitente,Mensaje
0,2025-01-25,12:01:53,Angela,NaN
1,2025-01-25,12:01:53,Angela,NaN
2,2025-01-25,12:01:50,Angela,NaN
3,2025-01-25,12:03:07,Angela,holi
4,2025-01-25,12:03:14,Angela,marce
...,...,...,...,...
48041,2025-06-23,02:55:19,Marcello,es minimalista
48042,2025-06-23,02:55:43,Marcello,solo siento que difiere mucho con la primera d...
48043,2025-06-23,02:55:49,Marcello,amor
48044,2025-06-23,02:55:57,Marcello,hay un 4to metatarsiano también?


In [ ]:
import pandas as pd
from pysentimiento import create_analyzer
from tqdm import tqdm

# Cargar datos
df = pd.read_csv("data/chat_limpio.csv")
df = df.dropna(subset=["Mensaje"])
df = df[df["Mensaje"].str.strip() != ""]

# Crear el analizador una vez
analyzer = create_analyzer(task="emotion", lang="es")

# Configurar lotes
batch_size = 100  # puedes ajustar según tu RAM/CPU
mensajes = df["Mensaje"].tolist()
emociones = []

# Procesar por lotes
for i in tqdm(range(0, len(mensajes), batch_size)):
    batch = mensajes[i:i+batch_size]
    resultados = analyzer.predict(batch)
    emociones.extend([r.output for r in resultados])

# Agregar resultados al DataFrame
df["Sentimiento"] = emociones

# Guardar resultados
df.to_csv("data/chat_limpio_con_emociones.csv", index=False)
